In [1]:
!nvidia-smi

Mon Jul 17 16:03:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q transformers datasets

# "The tokenizers that depend on the SentencePiece library will not be available with a standard transformers installation"
!pip install -q sentencepiece

# Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`:
# Please run `pip install transformers[torch]` or `pip install accelerate -U`
!pip install -q transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import nltk
import torch

from matplotlib import pyplot as plt
from nltk.tokenize import sent_tokenize
from transformers import (
    pipeline,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    set_seed)
from datasets import load_dataset, load_from_disk, load_metric
from tqdm.auto import tqdm

In [4]:
# gpu check in torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
model_ckpt = "google/pegasus-cnn_dailymail"

# Load the tokenizer associated with the Pegasus model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=False)

# Instantiate the Pegasus model for sequence-to-sequence tasks
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

## Train with pretrained model in custom data

In [6]:
# download and unzip data
!wget -q https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip -q summarizer-data.zip

In [7]:
dataset_loader = load_from_disk('samsum_dataset')
dataset_loader

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [8]:
print(f"Features: {dataset_loader['train'].features}")

Features: {'id': Value(dtype='string', id=None), 'dialogue': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}


In [9]:
split_rows = [dataset_loader[split].num_rows for split in dataset_loader]
split_rows

[14732, 819, 818]

In [10]:
# sample data
print("Original Dialog::::::")
print(dataset_loader["train"][1]["dialogue"])
print("*"*10)
print("Summary::::::")
print(dataset_loader["train"][1]["summary"])

Original Dialog::::::
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great
**********
Summary::::::
Olivia and Olivier are voting for liberals in this election. 


In [11]:
def convert_to_features(batch_data):
  # Tokenize the dialogue in the batch using the tokenizer
  input_encodings = tokenizer(batch_data['dialogue'], max_length=1024, truncation=True)

  # Tokenize the summary in the batch using the tokenizer as the target tokenizer
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(batch_data['summary'], max_length=128, truncation=True)

  # Return the converted features as a dictionary
  return {
      'input_ids': input_encodings['input_ids'],         # Input token IDs
      'attention_mask': input_encodings['attention_mask'],  # Attention mask
      'labels': target_encodings['input_ids']             # Target token IDs for the model's training
  }


In [12]:
dataset_batch_preprocess = dataset_loader.map(convert_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [13]:
dataset_batch_preprocess

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [14]:
sample_data = dataset_batch_preprocess['train'][1]
print(f"id: {sample_data['id']}")
print("*"*30)
print(f"dialogue length::{len(sample_data['dialogue'].split())} \n{sample_data['dialogue']}")
print("*"*30)
print(f"summary length::{len(sample_data['summary'].split())} \n{sample_data['summary']}")
print("*"*30)
print(f"input_ids length::{len(sample_data['input_ids'])} \n{sample_data['input_ids']}")
print("*"*30)
print(f"attention_mask length::{len(sample_data['attention_mask'])} \n{sample_data['attention_mask']}")
print("*"*30)
print(f"labels length::{len(sample_data['labels'])} \n{sample_data['labels']}")

id: 13728867
******************************
dialogue length::18 
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great
******************************
summary length::10 
Olivia and Olivier are voting for liberals in this election. 
******************************
input_ids length::26 
[18038, 151, 2632, 127, 119, 6228, 118, 115, 136, 2974, 152, 10463, 151, 35884, 130, 329, 107, 18038, 151, 2587, 314, 1242, 10463, 151, 1509, 1]
******************************
attention_mask length::26 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
******************************
labels length::12 
[18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]


In [15]:
print(f"target decoding: {tokenizer.decode(sample_data['labels'])}")
print(f"input decoding: {tokenizer.decode(sample_data['input_ids'])}")
print(f"attention mas decoding: {tokenizer.decode(sample_data['attention_mask'])}")

target decoding: Olivia and Olivier are voting for liberals in this election.</s>
input decoding: Olivia: Who are you voting for in this election? Oliver: Liberals as always. Olivia: Me too!! Oliver: Great</s>
attention mas decoding: </s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


## Evaluating pretrained model

*   num_beams refers to beam search, which is used for text generation. It returns the n most probable next words, rather than greedy search which returns the most probable next word.


In [16]:
gen_kwargs = {"length_penalty": 0.7, "num_beams":8, "max_length": 32}
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")

sample_text = dataset_loader["test"][0]["dialogue"]
reference_summary = dataset_loader["test"][0]["summary"]

print("Dialogue:")
print(sample_text)
print('-'*25)
print("\nReference Summary:")
print(reference_summary)
print("\nModel Summary:")
model_summary = pipe(sample_text, **gen_kwargs)
print(model_summary[0]["summary_text"])

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him 


## Training

In [17]:
from transformers import (DataCollatorForSeq2Seq,
                          TrainingArguments,
                          Trainer)

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=3,
    warmup_steps=500, per_device_train_batch_size=1,
    per_device_eval_batch_size=1, weight_decay=0.001,
    logging_steps=10, evaluation_strategy='steps',
    eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [18]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [19]:
# optimizer = torch.optim.AdamW(model_pegasus.parameters())
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_batch_preprocess["test"],
                  eval_dataset=dataset_batch_preprocess['validation'])

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=153, training_loss=2.5962425930048125, metrics={'train_runtime': 463.4581, 'train_samples_per_second': 5.301, 'train_steps_per_second': 0.33, 'total_flos': 938892521349120.0, 'train_loss': 2.5962425930048125, 'epoch': 2.99})

In [20]:
def generate_batch_sized_chunks(list_of_elements, batch_size: int):
  '''G
  '''
  for index in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[index: index + batch_size]


### ROUGE score
ROUGE stands for "Recall-Oriented Understudy for Gisting Evaluation" and is a set of metrics commonly used in natural language processing and text summarization tasks to evaluate the quality of automatic summaries compared to reference summaries.

ROUGE scores measure the overlap between the n-grams (contiguous sequences of n words) in the generated summary and the reference summary. The most commonly used variants of ROUGE include ROUGE-N, which measures the overlap of n-grams, and ROUGE-L, which considers the longest common subsequence between the summaries.

To calculate ROUGE scores, you need both the generated summary and the reference summary. The ROUGE metrics compute precision, recall, and F1-score, providing an indication of the quality of the generated summary.

In [21]:
def calculate_metric(dataset,metric, model,
                     tokenizer, batch_size=16,
                     device=device,
                     column_text='articles',
                     column_summary='highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches= list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches)):

    inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                    padding="max_length", return_tensors="pt")

    summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                attention_mask=inputs["attention_mask"].to(device),
                                length_penalty=0.8, # parameter for length penalty ensures that the model does not generate sequences that are too long.
                                num_beams=8,
                                max_length=128)

    # Finally, we decode the generated texts,
    # replace the  token, and add the decoded texts with the references to the metric.
    decoded_summaries = [tokenizer.decode(summary, skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
                          for summary in summaries]

    decoded_summaries = [summary.replace("", " ") for summary in decoded_summaries]

    metric.add_batch(predictions=decoded_summaries, references=target_batch)

  #  Finally compute and return the ROUGE scores.
  score = metric.compute()
  return score

In [22]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [23]:



rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-23-5a43aadd1b0e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [24]:

score = calculate_metric(
    dataset_loader['test'][0:10],
    rouge_metric,
    trainer.model, tokenizer, batch_size = 2,
    column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )


0it [00:00, ?it/s]

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.025314,0.0,0.025104,0.025188


In [ ]:
like always something different

In [25]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [26]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json')

In [27]:
# Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [28]:
# Prediction
sample_text = dataset_loader["test"][0]["dialogue"]
reference_summary = dataset_loader["test"][0]["summary"]

print("Dialogue:")
print(sample_text)
print('-'*25)
print("\nReference Summary:")
print(reference_summary)

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [29]:

gen_kwargs = {"length_penalty": 0.4, "num_beams":8, "max_length": 32}
pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)

print("\nModel Summary:")
model_summary = pipe(sample_text, **gen_kwargs)
print(model_summary[0]["summary_text"])


Model Summary:
Hannah is looking for Betty's number. Amanda can't find it. Larry called Hannah the last time they were at the park together. Hannah
